In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#page = driver.get("https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa")

page_url = 'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa'
page = requests.get(page_url)

In [3]:
content = page.content
soup = BeautifulSoup(content, features='html.parser')

span =soup.find_all('span')
print(len(span))

263


In [4]:
spans = soup.find_all("span", class_="css-rmqm02 eclomwz0")
addreses = soup.find_all("span", class_="css-17o293g es62z2j9")
spans[:4]

[<span class="css-rmqm02 eclomwz0">631 800 zł</span>,
 <span class="css-rmqm02 eclomwz0"><strong>11 998 zł/m²</strong></span>,
 <span class="css-rmqm02 eclomwz0">3 pokoje</span>,
 <span class="css-rmqm02 eclomwz0">52.66 m²</span>]

In [5]:
addreses[:5]

[<span class="css-17o293g es62z2j9">Warszawa, Białołęka, Przejezdna</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Wola, ul. Mordechaja Anielewicza 47</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Ursus, Karola Taylora</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Mokotów, Służewiec, Nowoursynowska</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Śródmieście, Emilii Plater</span>]

In [6]:
spans[3:9]

[<span class="css-rmqm02 eclomwz0">52.66 m²</span>,
 <span class="css-rmqm02 eclomwz0">553 000 zł</span>,
 <span class="css-rmqm02 eclomwz0"><strong>14 946 zł/m²</strong></span>,
 <span class="css-rmqm02 eclomwz0">2 pokoje</span>,
 <span class="css-rmqm02 eclomwz0">37 m²</span>,
 <span class="css-rmqm02 eclomwz0">599 900 zł</span>]

In [7]:
price = spans[::4]
rooms = spans[2::4]
space = spans[3::4]

In [8]:
import re

def clean(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext  = re.sub(cleanr, '', str(raw_html))
    cleantext= re.sub("[^0-9]", "", cleantext)
    return cleantext 

cleaned_price = []

for p in price:
    cleaned = clean(p)
    cleaned_price.append(cleaned)
    
cleaned_price[:5]

['631800', '553000', '599900', '1285000', '899000']

In [9]:
[clean(p) for p in price][:5]

['631800', '553000', '599900', '1285000', '899000']

In [10]:
cleaned_rooms =[]

for r in rooms:
    cleaned = clean(r)
    cleaned_rooms.append(cleaned)
cleaned_rooms[:5]

['3', '2', '2', '3', '2']

In [11]:
space[:5]

[<span class="css-rmqm02 eclomwz0">52.66 m²</span>,
 <span class="css-rmqm02 eclomwz0">37 m²</span>,
 <span class="css-rmqm02 eclomwz0">44.69 m²</span>,
 <span class="css-rmqm02 eclomwz0">110 m²</span>,
 <span class="css-rmqm02 eclomwz0">56 m²</span>]

In [12]:
cleaned_space =[]

for s in space:
    s = str(s)
    cleaned_space.append(s[34:40].replace('m','').replace('<',"").replace('²','').replace(' ',''))

cleaned_space[:5]

['52.66', '37', '44.69', '110', '56']

In [13]:
addreses[:5]

[<span class="css-17o293g es62z2j9">Warszawa, Białołęka, Przejezdna</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Wola, ul. Mordechaja Anielewicza 47</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Ursus, Karola Taylora</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Mokotów, Służewiec, Nowoursynowska</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Śródmieście, Emilii Plater</span>]

In [14]:
list = []

for adr in addreses:
    a = str(adr)
    list.append(a.replace('<span class="css-17o293g es62z2j9">', '').replace('</span>', ''))

In [15]:
list[:5]

['Warszawa, Białołęka, Przejezdna',
 'Warszawa, Wola, ul. Mordechaja Anielewicza 47',
 'Warszawa, Ursus, Karola Taylora',
 'Warszawa, Mokotów, Służewiec, Nowoursynowska',
 'Warszawa, Śródmieście, Emilii Plater']

In [16]:
df = pd.DataFrame({'Address': list, 'Price': cleaned_price, 'Space': cleaned_space, 'Rooms': cleaned_rooms})
df

,Address,Price,Space,Rooms
0,"Warszawa, Białołęka, Przejezdna",631800,52.66,3
1,"Warszawa, Wola, ul. Mordechaja Anielewicza 47",553000,37,2
2,"Warszawa, Ursus, Karola Taylora",599900,44.69,2
3,"Warszawa, Mokotów, Służewiec, Nowoursynowska",1285000,110,3
4,"Warszawa, Śródmieście, Emilii Plater",899000,56,2
5,"Warszawa, Mokotów, Bełdan",499000,46.6,3
6,"Warszawa, Mokotów, Dolny Mokotów, ul. Bartycka",840000,55.92,3
7,"Warszawa, Praga-Południe, Witolin, Roztocka",645000,43.01,2
8,"Warszawa, Mokotów, Dolny Mokotów, ul. Bartycka",925000,61.86,3
9,"Warszawa, Mokotów, Dolny Mokotów, ul. Bartycka",794622,61.89,3


In [17]:
df.Address[2]

'Warszawa, Ursus, Karola Taylora'

In [18]:
"https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=" + str(1)

'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=1'

In [19]:
def cleaned_space(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    s = str(string)
    return (s[34:40].replace('m','').replace('<',"").replace('²','').replace(' ','')) 

def clean_without_coma(raw_html):
   """clean numbers froim html 

   Args:
       raw_html (beautiful suoup object): _description_

   Returns:
       int: scrapped number
   """
   cleanr = re.compile('<.*?>')
   cleantext  = re.sub(cleanr, '', str(raw_html))
   cleantext= re.sub("[^0-9]", "", cleantext)
   return cleantext 
   
def cleaned_addreses(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    a = str(string)
    return a.replace('<span class="css-17o293g es62z2j9">', '').replace('</span>', '')

In [20]:
addreses=[] #List of flats adresses
out_prices=[] #List house prices
out_space=[] #List of house space
out_rooms=[] #List of number of rooms in flat

page_url = "https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page="

for number in range(5):
    page = requests.get(page_url +str(number))
    content = page.content
    soup = BeautifulSoup(content, features='html.parser')

    spans = soup.find_all("span", class_="css-rmqm02 eclomwz0")
    addres = soup.find_all("span", class_="css-17o293g es62z2j9")
    price = spans[::4]
    rooms = spans[2::4]
    space = spans[3::4]

    clean_price = [clean(p) for p in price]
    clean_rooms = [clean(r) for r in rooms]
    clean_space = [cleaned_space(s) for s in space]
    clean_addres = [cleaned_addreses(a) for a in addres]

    addreses += clean_addres
    out_prices += clean_price
    out_space += clean_space
    out_rooms += clean_rooms






In [21]:
cleaned_space(space[0])

'51'

In [22]:
clean_addres[:5]

['Warszawa, Wilanów, aleja rzeczypospolitej 33',
 'Warszawa, Wilanów, ul. Goplańska 29',
 'Warszawa, Mokotów, Czerniaków, ul. Zwierzyniecka 17',
 'Warszawa, Mokotów, Domaniewska 47',
 'Warszawa, Białołęka, Brzeziny, Przyrodnicza 11']

In [23]:
clean_space[:5]

['51', '50', '57', '113.92', '62']

In [24]:
df = pd.DataFrame({'Address': addreses, 'Price': out_prices, 'Space': out_space, 'Rooms': out_rooms})
#df.to_csv('listings.csv', index=False, encoding='utf-8')

In [25]:
df

,Address,Price,Space,Rooms
0,"Warszawa, Mokotów",450000,28.63,1
1,"Warszawa, Rembertów, Stary Rembertów",390000,35.17,2
2,"Warszawa, Rembertów, al. Aleja gen. Antoniego ...",511801,39.43,2
3,"Warszawa, Mokotów, Służewiec, Nowoursynowska",1285000,110,3
4,"Warszawa, Śródmieście, Emilii Plater",899000,56,2
...,...,...,...,...
190,"Warszawa, Wola, al. Prymasa Tysiąclecia",1795000,90.23,4
191,"Warszawa, Mokotów, Wiartel 8",649000,50,3
192,"Warszawa, Centrum, ul. gen. Władysława Andersa",334040,21,1
193,"Warszawa, Praga-Południe, ul. Chodakowska",,34.54,2


In [26]:
df[:5]

,Address,Price,Space,Rooms
0,"Warszawa, Mokotów",450000,28.63,1
1,"Warszawa, Rembertów, Stary Rembertów",390000,35.17,2
2,"Warszawa, Rembertów, al. Aleja gen. Antoniego ...",511801,39.43,2
3,"Warszawa, Mokotów, Służewiec, Nowoursynowska",1285000,110,3
4,"Warszawa, Śródmieście, Emilii Plater",899000,56,2


In [27]:
soup.find_all("a", href = True)[1]

<a class="css-zygftp e1ia8j2v0" href="https://pomoc.otodom.pl/hc/pl/articles/203136942-Co-wp%C5%82ywa-na-kolejno%C5%9B%C4%87-og%C5%82osze%C5%84-w-wynikach-wyszukiwania-" rel="noreferrer" target="_blank">Jak pozycjonowane są ogłoszenia?</a>

In [28]:
inner_page_url = "https://www.otodom.pl"

for a in soup.find_all('a', href=True):
    print(inner_page_url + a['href'])

https://www.otodom.pl/
https://www.otodom.plhttps://pomoc.otodom.pl/hc/pl/articles/203136942-Co-wp%C5%82ywa-na-kolejno%C5%9B%C4%87-og%C5%82osze%C5%84-w-wynikach-wyszukiwania-
https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=1&limit=36&market=ALL&locations=%5Bcities_6-26%5D&isPromoted=true
https://www.otodom.pl/pl/oferta/dwupokojowe-garderoba-garaz-ogrodek-820tys-ID4fH3C
https://www.otodom.pl/pl/oferta/2-pok-oddzielna-kuchnia-garderoba-bezposrednio-ID4eNUx
https://www.otodom.pl/pl/oferta/mieszkanie-w-zieleni-zwierzyniecka-w-warszawie-ID4hhIS
https://www.otodom.pl/pl/oferta/5-pokojowe-mieszkanie-113m2-2-balkony-ID4dcCr
https://www.otodom.pl/pl/oferta/bezposrednio-nowe-bezczynszowe-ogrodek-3-pokoje-ID4gwmG
https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-60m2-loggia-ID4fVaT
https://www.otodom.pl/pl/oferta/mieszkanie-z-pieknym-ogrodkiem-sarmacka-ID4hrDH
https://www.otodom.pl/pl/oferta/sprzedam-mieszkanie-inwestycyjne-57m-zoliborz-ID4fUCF
https://www.otodom.pl/pl/ofert

In [29]:
inner_page_url = "https://www.otodom.pl"

In [30]:
a = soup.find_all('a', href=True)[2:41]

In [31]:
type(soup)

bs4.BeautifulSoup

In [32]:
a[5]['href']

'/pl/oferta/bezposrednio-nowe-bezczynszowe-ogrodek-3-pokoje-ID4gwmG'

In [33]:
for a in soup.find_all('a', href=True)[2:41]:
    print(inner_page_url + a['href'])

https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=1&limit=36&market=ALL&locations=%5Bcities_6-26%5D&isPromoted=true
https://www.otodom.pl/pl/oferta/dwupokojowe-garderoba-garaz-ogrodek-820tys-ID4fH3C
https://www.otodom.pl/pl/oferta/2-pok-oddzielna-kuchnia-garderoba-bezposrednio-ID4eNUx
https://www.otodom.pl/pl/oferta/mieszkanie-w-zieleni-zwierzyniecka-w-warszawie-ID4hhIS
https://www.otodom.pl/pl/oferta/5-pokojowe-mieszkanie-113m2-2-balkony-ID4dcCr
https://www.otodom.pl/pl/oferta/bezposrednio-nowe-bezczynszowe-ogrodek-3-pokoje-ID4gwmG
https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-60m2-loggia-ID4fVaT
https://www.otodom.pl/pl/oferta/mieszkanie-z-pieknym-ogrodkiem-sarmacka-ID4hrDH
https://www.otodom.pl/pl/oferta/sprzedam-mieszkanie-inwestycyjne-57m-zoliborz-ID4fUCF
https://www.otodom.pl/pl/oferta/mieszkanie-53m-19-ta-dzielnica-garaz-komorka-ID4gOjU
https://www.otodom.pl/pl/oferta/inwestycyjnie-swietna-lokalizacja-przy-wum-ID4h9OD
https://www.otodom.pl/pl/oferta/skand

In [34]:
url = 'https://www.otodom.pl/pl/oferta/2-pokoje-13-min-do-metra-garaz-i-komorka-ID4h80u'

page = requests.get(url)
content = page.content
soup = BeautifulSoup(content, features='html.parser')

data= soup.find_all("div", class_="css-1wi2w6s estckra5")

"""market = data[10]
year = data[12]
elevator = data[15]s
balcony = data[5]
level = data[4]
parking_place =data[7]"""

'market = data[10]\nyear = data[12]\nelevator = data[15]s\nbalcony = data[5]\nlevel = data[4]\nparking_place =data[7]'

In [35]:
data[:15]

[<div class="css-1wi2w6s estckra5">52,9 m²</div>,
 <div class="css-1wi2w6s estckra5">pełna własność</div>,
 <div class="css-1wi2w6s estckra5">2 </div>,
 <div class="css-1wi2w6s estckra5">do zamieszkania</div>,
 <div class="css-1wi2w6s estckra5">4/6</div>,
 <div class="css-1wi2w6s estckra5">balkon</div>,
 <div class="css-1wi2w6s estckra5">700 zł</div>,
 <div class="css-1wi2w6s estckra5">garaż/miejsce parkingowe</div>,
 <div class="css-1wi2w6s estckra5">tak</div>,
 <div class="css-1wi2w6s estckra5">miejskie</div>,
 <div class="css-1wi2w6s estckra5">wtórny</div>,
 <div class="css-1wi2w6s estckra5">biuro nieruchomości </div>,
 <div class="css-1wi2w6s estckra5">2003</div>,
 <div class="css-1wi2w6s estckra5">blok</div>,
 <div class="css-1wi2w6s estckra5">drewniane</div>]

In [36]:
print(year)
clean(year)

NameError: name 'year' is not defined

In [ ]:
print(level)
clean(level)

<div class="css-1wi2w6s estckra5">4/6</div>


'46'

In [ ]:
str(data[4])[34:-6]

'4/6'

In [ ]:
print(market)
clean(market)

<div class="css-1wi2w6s estckra5">wtórny</div>


''

In [ ]:
str(data[10])[34:-6]

'wtórny'

In [ ]:
print(elevator)
clean(elevator)

<div class="css-1wi2w6s estckra5">tak</div>


''

In [ ]:
str(data[15])[34:-6]

'tak'

In [ ]:
str(data[5])[34:-6]

'balkon'

In [ ]:
str(data[7])[34:-6]

'garaż/miejsce parkingowe'

In [133]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


def clean(raw_html):
   """clean numbers froim html 

   Args:
       raw_html (beautiful suoup object): _description_

   Returns:
       int: scrapped number
   """
   cleanr = re.compile('<.*?>')
   cleantext  = re.sub(cleanr, '', str(raw_html))
   cleantext= re.sub("[^0-9]", "", cleantext)
   return cleantext 

def cleaned_addreses(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    a = str(string)
    return a.replace('<span class="css-17o293g es62z2j9">', '').replace('</span>', '')


def cleaned_space(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    s = str(string)
    return (s[34:40].replace('m','').replace('<',"").replace('²','').replace(' ','')) 

def cleaned_elevator(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    s = str(string)
    return (s[34:40].replace('m','').replace('<',"").replace('²','').replace(' ','')) 

def scrap_data():
    """scrap data

    Returns:
        pandas_data_frame: four colums
    """
    addreses = [] #List of flats adresses
    out_prices = [] #List house prices
    out_space = [] #List of house space
    out_rooms = [] #List of number of rooms in flat
    out_market = []
    out_year = []
    out_elevator = []
    out_balcony = []
    out_level = []
    out_parking_place = []

    page_url = "https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page="

    for number in range(5):
        page = requests.get(page_url +str(number))
        content = page.content
        soup = BeautifulSoup(content, features='html.parser')

        spans = soup.find_all("span", class_="css-rmqm02 eclomwz0")
        addres = soup.find_all("span", class_="css-17o293g es62z2j9")
        price = spans[::4]
        rooms = spans[2::4]
        space = spans[3::4]

        clean_price = [clean(p) for p in price]
        clean_rooms = [clean(r) for r in rooms]
        clean_space = [cleaned_space(s) for s in space]
        clean_addres = [cleaned_addreses(a) for a in addres]

        market, year, elevator, balcony, level, parking_place = inner_page_data_scrap(soup)

        addreses += clean_addres
        out_prices += clean_price
        out_space += clean_space
        out_rooms += clean_rooms
        out_market += market
        out_year += year
        out_elevator += elevator
        out_balcony += balcony
        out_level += level
        out_parking_place += parking_place
    
    print(out_market, out_year, out_elevator,out_balcony,out_level,out_parking_place)
    print(len(out_market),len(out_year),len(out_elevator),len(out_balcony),len(out_level),len(parking_place),len(out_space))
    df = pd.DataFrame({
        'Address': addreses, 'Price': out_prices, 'Space': out_space, 'Rooms': out_rooms, 'Market': out_market, 'Year': out_year, 'Evelevator': out_elevator, 'Balcony': out_balcony, 'Level': out_level, 'Parking place': out_parking_place
        })
    return df

def inner_page_data_scrap(soup):
    """scraps detail from inner html page of an offert

    Args:
        soup (bs4.BeautifulSoup): current html page of offerts list

    Returns:
       market, year, elevator, balcony, level, parking_place (list): _description_
    """

    inner_page_url = "https://www.otodom.pl"
    market = []
    year = []
    elevator = []
    balcony = []
    level = []
    parking_place = []
    pages_url = []
    for a in soup.find_all('a', href=True)[3:41]:
        pages_url.append(inner_page_url + a['href'])

    for u in pages_url:
        if len(u) < 5:
            pass 
        else:
            page = requests.get(u)
            content = page.content
            soup = BeautifulSoup(content, features='html.parser')
            data= soup.find_all("div", class_="css-1qzszy5 estckra8")

            try:
                market.append(str(data[21].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
            except IndexError:
                market.append('')
            try:
                year.append(str(data[25].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7]) #clean(data[12])
            except IndexError:
                year.append('')
            try:
                elevator.append(str(data[31].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
            except IndexError:
                elevator.append('')
            try:
                balcony.append(str(data[11].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
            except IndexError:
                balcony.append('')
            try:
                level.append(str(data[9].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
            except IndexError:
                level.append('')
            try:
                parking_place.append(str(data[15].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
            except IndexError:
                parking_place.append('')


    return market, year, elevator, balcony, level, parking_place



data = scrap_data()

['wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'pierwotny', 'pierwotny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'pierwotny', '', 'wtórny', '', '', '', '', '', '', '', '', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', '', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'pierwotny', 'pierwotny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'pierwotny', '', 'wtórny', '', '', '', '', '', '', '', 'wtórny', 'wtórny', 'wtórny', '', 'wtórny', 'wtórny', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'wtórny', 'wtórny', 'wtórny', '', 'wtórny', '', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', '', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', '', '', 'wtórny', 'wtórny', 'pierwotny', 'pierw

ValueError: All arrays must be of the same length

In [ ]:
data

In [131]:
market = []
year = []
elevator = []
balcony = []
level = []
parking_place = []
pages_url = []
u = 'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=1'
page = requests.get(u)
content = page.content
soup = BeautifulSoup(content, features='html.parser')

for a in soup.find_all('a', href=True)[3:41]:
    pages_url.append(inner_page_url + a['href'])

for u in pages_url:
    if len(u) < 5:
       pass 
    else:
        page = requests.get(u)
        content = page.content
        soup = BeautifulSoup(content, features='html.parser')
        data= soup.find_all("div", class_="css-1qzszy5 estckra8")

        try:
            market.append(str(data[21].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
        except IndexError:
            market.append('')
        try:
            year.append(str(data[25].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7]) #clean(data[12])
        except IndexError:
            year.append('')
        try:
            elevator.append(str(data[31].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
        except IndexError:
            elevator.append('')
        try:
            balcony.append(str(data[11].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
        except IndexError:
            balcony.append('')
        try:
            level.append(str(data[9].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
        except IndexError:
            level.append('')
        try:
            parking_place.append(str(data[15].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
        except IndexError:
            parking_place.append('')

In [130]:
try:
    market.append(str(data[21].find_all('div',class_="css-1wi2w6s estckra5"))[35:-7])
except IndexError:
    market.append('')

empty


In [127]:
len(data)

20

In [122]:
str(data[21].find_all('div',class_="css-1wi2w6s estckra5"))

IndexError: list index out of range

In [132]:
print(market, year, elevator,balcony,level,parking_place)

['wtórny', 'pierwotny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'pierwotny', 'pierwotny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'wtórny', 'pierwotny', '', 'wtórny', '', '', '', '', '', '', '', '', '', 'wtórny'] ['1973', 'apartamentowiec', '2010', '1989', '2022-05-19', '2002', '2002', '2000', '1910', '2000', '2022-06-10', '2022', '2009', '2006', '2022', '2020', '2015', '2000', '2022', 'blok', '1983', '2005', '2020', '1974', '1976', '2021', '', 'tak', '', '', '', '', '', '', '', '', '', '1937'] ['meble', '', 'tak', 'nie', 'tak', 'monitoring / ochrona', 'telewizja kablowa, internet, telefon', 'telewizja kablowa, internet', 'nie', 'domofon / wideofon', 'plastikowe', 'tak', 'tak', 'tak', 'tak', 'tak', 'tak', 'nie', 'tak', 'monitoring / ochrona', 'tak', 'telewizja kablowa, internet, telefon', 'domofon / wideofon, monitoring / ochrona', 'nie', 'tak', 'tak', '', 'pom. użytk

In [134]:
l = ['', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', '', '', '', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', '', '', '', '', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', '', '', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', '', '', '', '', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', '', '', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', '', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe', 'garaż/miejsce parkingowe']

In [135]:
len(l)

190